### HEO Model : 1 Battery-1 Genset. 
=========================================================================================================================
$ 
\\
$

$ 
\\
$

### Optimization parameters.


$ Q_{\mathrm{max}} \hspace{1cm}$ : Maximal energy charge stored in battery (kWh).



$ Q_{\mathrm{init}} \hspace{1.1cm}$ : Initial energy charge stored in battery (kWh).



$ Q_{\mathrm{final}} \hspace{0.85cm}$ :  Final charge stored on battery (kWh).



$ P_{\mathrm{max}\;} \hspace{0.85cm}$ : Maximal power can be generated by genset (kW).



$ P_{\mathrm{min}\;} \hspace{0.9cm}$ : Minimal power can be generated by genset (kW).



${FC}_{max} \hspace{0.7cm}$ : Maximal fuel consumption (g).



$ {FC}_{offset} \hspace{0.4cm}$ : Offset for fuel consumption model (at P=0).



$ a \hspace{1.7cm}$ : Slope from fuel consumption model.



$ b \hspace{1.7cm}$ : Intercept from fuel consumption model.



$ \Delta t \hspace{1.5cm}$ : Time step in discretization (h).




$ t_{\mathrm{max}\;} \hspace{1.1cm}$ : Max time in simulation (h).



$ n \hspace{1.7cm}$ :  Number of time steps in simulation.



$ t_{\mathrm{max}\;} \hspace{1.1cm}$ : Max time in simulation (h).



$ t \hspace{1.8cm}$ :  Time vector in simulation (dim(t)=n).



$ \eta^{\mathrm{toBat}} \; \hspace{0.9cm}$ :  Efficiency when charging the battery.



$ \eta^{\mathrm{fromBat}} \; \hspace{0.5cm}$ :   Efficiency when producing from the battery.



$ L \hspace{1.6cm}$ : Vector of pre-defined load profile (must be of same length as time vector, dim(L)=dim(t)=n) (kW).

$\newline$ 


### Optimization model.

$\mathrm{minimize}\;\;\;\;\;\sum_{k=1}^n {FC}_{k\;} \cdot \frac{\Delta t}{1000}+K_{\mathrm{start}} \sum_{k=2}^n z_k$  

$\newline$ 

Fuel consumption model 1 (linear):     $\hspace{1cm} {FC}_{k\;} ={aP}_{k\;} +b-{FC}_{offset} \cdot \left(1-y_k \right)\hspace{3cm}$      $k=1,\dots ,n$

Load requirements:     $\hspace{3.5cm}L_k =P_k^{\mathrm{load}} +\eta {\;}^{\mathrm{fromBat}} {\cdot \;P}_k^{\mathrm{fromBat}}\hspace{4.2cm}$      $k=1,\dots ,n$  


Power split:   $\hspace{4.8cm}P_{k\;} =P_k^{\mathrm{load}} {+\;P}_k^{\mathrm{toBat}}\hspace{6.3cm}$   $k=1,\dots ,n$

Charge balance (initial):   $\hspace{2.8cm}Q_0 =Q_{\mathrm{init}}\hspace{6.3cm}$  

Charge balance:  $\hspace{4cm}Q_k =Q_{k-1} +\eta^{\mathrm{toBat}} \cdot \;P_k^{\mathrm{toBat}} \Delta t-\;P_k^{\mathrm{fromBat}} \Delta t\hspace{1.8cm}$     $k=1,\dots ,n$ 


Charge balance (final):     $\hspace{3cm}Q_n =Q_{\mathrm{final}}\hspace{3cm}$   


Logical conditions on genset: $\hspace{2cm}P_{k\;} \le {0\ldotp 9P}_{\mathrm{max}\;} {\cdot y}_k\hspace{6.7cm}$    $k=1,\dots ,n$ 


(0 or in 0.2P_max - 0.9Pmax):  $\hspace{2cm}P_{k\;} \le {0\ldotp 2P}_{\mathrm{max}\;} {\cdot y}_k\hspace{6.9cm}$    $k=1,\dots ,n$




Logical conditions on battery:   $\hspace{1.9cm}y_k^{\mathrm{toBat}} +y_{k\;}^{\mathrm{fromBat}} \le 1\hspace{6.4cm}$     $k=1,\dots ,n$  $ \\$     
$\hspace{6.9cm}P_k^{\mathrm{toBat}} \le 0\ldotp 9P_{\mathrm{max}} {\cdot y}_k^{\mathrm{toBat}}\hspace{5.7cm}$ $k=1,\dots ,n$ 


$\hspace{6.9cm}P_k^{\mathrm{fromBat}} \le 0\ldotp 9P_{\mathrm{max}} {\cdot \;y}_k^{\mathrm{fromBat}}\hspace{4.8cm}$  $k=1,\dots ,n$ 

$\newline$ 


Constraints for linearization of objective:  $\hspace{0.5cm}z_k \ge y_k -y_{k-1}\hspace{7.0cm}$ $k=2,\ldotp \ldotp \ldotp ,n$

$\newline$ 

Variable bounds:  $\hspace{4.1cm}0\ldotp 2Q_{\mathrm{max}} \le Q_k \le Q_{\mathrm{max}}\hspace{5.8cm}$ $k=0,\ldotp \ldotp \ldotp ,n$  $ \\$  
$\hspace{7cm}0\le P_k \le 0\ldotp 9P_{\mathrm{max}}\hspace{6.5cm}$  $k=1,\ldotp \ldotp \ldotp ,n$    
 
$\hspace{7cm}0\le P_k^{\mathrm{toBat}} ,P_k^{\mathrm{fromBat}} \le {0\ldotp 9P}_{\mathrm{max}}\hspace{4.2cm}$ $k=1,\ldotp \ldotp \ldotp ,n$ 


$\hspace{7cm}0\le z_k \le 1\hspace{7.8cm}$  $k=2,\ldotp \ldotp \ldotp ,n$


$ \\$ Variable types:     $\hspace{1cm}Q_k , P_k^{\mathrm{toBat}} , P_k^{\mathrm{fromBat}} ,Z_k$ :Continuous $\hspace{1cm}P_k$  :Semi-continuous  $\hspace{1.5cm}y_k ,y_k^{\mathrm{toBat}} ,y_k^{\mathrm{fromBat}}$ :Binary

$\newline$ 

$ \\$ $P_k \hspace{2.1cm} $:  Power from genset during time step k
 
$ \\$ $P_k^{\mathrm{toBat}}\hspace{1.5cm}$: Power to charge battery during time step k
 
$ \\$ $P_k^{\mathrm{fromBat}}\hspace{1.2cm}$: Power delivered from battery during time step k
 
$ \\$ $Q_k\hspace{2.1cm}$: Energy charge on battery at the end of time step k
 
$ \\$ $y_k\hspace{2.3cm}$: Variable equal to 1 if genset is used in time step k, variable equal to 0 if genset not used in time step k

$ \\$ $y_k^{\mathrm{toBat}}\hspace{1.6cm}$: Variable equal to 1 if battery is charged under time step k

$ \\$ $y_k^{\mathrm{fromBat}}\hspace{1.3cm}$: Variable equal to 1 if battery if used for load in time step k

$ \\$ $z_k\hspace{2.3cm}$: Variable used to linearize additional cost when starting genset (  )



$\newline $
$\newline $




$\newline $
### Importing modules.

In [1]:
import sys
import numpy as np
import plotly.express as px
from IPython.display import Image
import pandas as pd
from pulp import *
from src.functions.fuel_consumption import fuelCon
from src.functions.load_window import lwd 

ModuleNotFoundError: No module named 'src.functions'

### Python environment.

In [ ]:
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

$\newline $
### Introducing problem Parameters.

In [ ]:
Q_max = 250                                                   # maximal capacity stored on battery.
Q_0 = 0.5*Q_max                                               # inital capacity stored on battery.
Q_final = 0.5*Q_max                                           # Final capacity stored on the battery. 
eff_to_bat = 0.98                                             # battery charging efficiency.
eff_from_bat = 0.98                                           # battery transfer efficiency. 
P_max = 1000
P_min = 0
                                                # max output from gen set A.
dt = 1                                                        # simulation time step dt.
t_max = 10                                                    # time span for simulation = t_max hours.
t = np.atleast_2d(np.arange(0,t_max,dt)).T.conj()             # time scale in hours.
n = len(t)                                                    # number of time steps.
m = 1                                                         # number of Gensets used on the trip.
fc_j_offset = 190                                             # genset j fuel consumption when no power gnerated (p=0) in g/h.
K_j_start = 70.0                                              # Addicitional starting cost.
V_steps = [x for x in range(0,n)]                            # Time steps vector. 
V_steps_z = V_steps[:-1]                                      # Time steps vector without the final step. 


L = lwd (200,50,5,                                           # Power requirement window.
                50,300,5,
                200,150,10,
                150,250,30,
                0,5,20,30)                                   # Pre-defined load porfile requirements framed in 60 time steps.

$\newline $
### Creating the problem variable. 

In [ ]:
Optim = LpProblem('Energy_Opt',LpMinimize)

$\newline $
### Slope, intercept and maximum fuel bound calculation.

In [ ]:
a_j = (fuelCon(0.9*P_max, P_max) - fuelCon(0.2*P_max, P_max)) /(0.9*P_max) #slope.
b_j = fuelCon(0.2*P_max, P_max) - a_j*0.2*P_max #Intercept.
maxFC = fuelCon(0.9*P_max,P_max) # Max fuel bound.
print('FC Linear model caracteristics')
print('-----------------------------')
print('Slope =', a_j)
print('Intercept =', b_j)

$\newline $
### Setting-up decision Variables.

In [ ]:
Q_bat = LpVariable.dicts("Q_bat", V_steps, lowBound=0.2*Q_max, upBound=Q_max, cat = LpContinuous)  # Battery charge at time step k.
P_From_bat = LpVariable.dicts("P_From_bat", V_steps, lowBound=P_min, upBound=0.9*P_max, cat = LpContinuous) # Power transfered from the battery to the load.
P = LpVariable.dicts("P", V_steps, lowBound=P_min, upBound=0.9*P_max, cat = LpContinuous)  # Power generated by the Genset A.
P_load = LpVariable.dicts("P_load", V_steps, lowBound=P_min, upBound=0.9*P_max, cat= LpContinuous) # Power transfered from the Genset A to the load at time step k.
Z =  LpVariable.dicts("Z", V_steps_z, lowBound=0, upBound=1, cat = LpInteger) # Aditional cost fuel oil consumption when starting Genset j.
FC = LpVariable.dicts("FC", V_steps, lowBound=0, upBound= maxFC, cat = LpContinuous) # Specific fuel oil consumption.
P_to_bat =  LpVariable.dicts("P_to_bat", V_steps, lowBound=P_min, upBound=0.9*P_max, cat = LpContinuous) 
Y_to_bat = LpVariable.dicts("Y_to_bat", V_steps, lowBound=0, upBound=1, cat = LpInteger) # Genset selecter to charge the battery at time step k. 
Y_from_bat = LpVariable.dicts("Y_from_bat", V_steps, lowBound=0, upBound=1, cat = LpInteger) # Battery selecter to transfert to the Genset j st time step k.
Y = LpVariable.dicts("Y", V_steps, lowBound=0, upBound=1, cat = LpInteger) # Genset selecter : work ==> Y=1, Not work ==> Y=0.

$\newline $
### Setting-up the objective function.

In [ ]:
FC = sum(FC[k] for k in V_steps) * dt/1000  # sum of the fuel oil comsumption for all gensets over all k steps.
L_added_cost = sum(Z[i] for i in V_steps_z) * K_j_start  # Sum of all of the additional costs including starting costs. 
Optim += lpSum (FC + L_added_cost), " Minimization fuel oil consumption objective " 
print(FC)

$\newline $
$\newline $
### Setting-up problem constraints.

In [ ]:

for k in V_steps:

  # Fuel oil consumption constraint.
  #Optim += FC[k] == P[k] * a_j + b_j - fc_j_offset * Y[k]     

  # Load requirements constraints
  Optim += P_load[k] + eff_from_bat*P_From_bat[k] == L[k]       
  Optim += P_load[k] + P_to_bat[k] == P[k]                  


  # Genset logical constraints.
  Optim += P[k]  <= 0.9 * P_max * Y[k]
  Optim += P[k]  >= 0.2 * P_max * Y[k]
  Optim += Y[k]  >= 0
  Optim += Y[k]  <= 1
    
  # Battery charging logical constraints.
  Optim += P_to_bat[k] <= 0.9 * P_max * Y_to_bat[k]
  Optim += P_From_bat[k] <= 0.9 * P_max * Y_from_bat[k]
  Optim += Y_to_bat[k] + Y_from_bat[k]  <= 1
  

  # Charge balance logical constraints.
  if k == V_steps[0] :                                            
    Optim += Q_bat[k] == Q_0 + eff_to_bat*P_to_bat[k]*dt - P_From_bat[k]*dt


  else :  
    Optim += Q_bat[k] == Q_bat[k-1] + eff_to_bat*P_to_bat[k]*dt - P_From_bat[k]*dt

    
# Additional starting costs constraint.
for k in range(V_steps[0], V_steps[-1]): 
  Optim += Z[k] >= Y[k + 1] - Y[k] 
  
# Charge balance at the Final time step.
Optim += Q_bat[V_steps[-1]] == Q_final

$\newline $
$\newline $
### Solving the problem.

In [ ]:
status = Optim.solve(GUROBI())

$\newline $
$\newline $
### Printing optimization status.

In [ ]:
LpStatus[status]

$\newline $
$\newline $
### Printing constraints per each time step.

In [ ]:
print(Optim.variables)

In [ ]:
for v in Optim.variables():
    print(v.name, "=", v.varValue )

$\newline $
$\newline $
### Calculating the optimized fuel comsumption.

In [ ]:
print("Total fuel comsumption of the trip:", value(Optim.objective),'g') 

$\newline $
### Pre-processing visualisation data.

In [ ]:
# Constructing list out of the P_A vector.

P_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('P_'+ str(i)):
      P_list.append(v.varValue)
print(P_list)

# Constructing list out of the P_A_load vector.

P_load_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('P_load_'+ str(i)):
      P_load_list.append(v.varValue)
print(P_load_list)

# Constructing list out of the P_A_to_bat vector.

P_to_bat_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('P_to_bat_'+ str(i)):
      P_to_bat_list.append(v.varValue)
print(P_to_bat_list)

# Constructing list out of the P_From_bat.

P_From_bat_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('P_From_bat_'+ str(i)):
      P_From_bat_list.append(v.varValue)
print(P_From_bat_list)

# Constructing list out of the Q_bat vector.

Q_bat_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('Q_bat_'+ str(i)):
      Q_bat_list.append(v.varValue)
print(Q_bat_list)

# constructing list out of the FC_A vector.

FC_list = []
for v in Optim.variables():
  for i in V_steps:
    if v.name == ('FC_A_'+ str(i)):
      FC_list.append(v.varValue/1000)
print(FC_list)

# Constructing list out of load profil vector.

d_load_list = []              
for i in V_steps:
    d_load_list.append(L[i])
print(d_load_list)



$\newline $

### Load requirement window. 

In [ ]:
L_dframe = {
             'Load requirement.': d_load_list,
             'Scale' : d_load_list}

L_df = pd.DataFrame(L_dframe)
fig = px.bar(L_df, x = V_steps,
             y = d_load_list,
             title = '________Typical Load Requirements.________',
             height = 400,
             width = 1350,
             labels = dict(x= "Time Step (h)", y = "Load demand (kwh)"), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps,L)

$\newline $
### Generated genset power.

In [ ]:
P_A_dframe = {
             'Power generated by the Genset.': P_list,
             'Scale' : P_list}

P_A_df = pd.DataFrame(P_A_dframe)
fig = px.bar(P_A_df, x = V_steps,
             y = P_list,
             title = '________Power Generated By The Genset A._________',
             height = 400,
             width = 1350,
             labels = dict(x = "Time Step (h)", y = "Power(kw)"), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, P_list)

$\newline $
### Realised power from the genset to load.

In [ ]:
P_to_load_dframe = {
             "Power from Genset to laod.": P_load_list,
             'Scale' : P_load_list}

P_to_load_df = pd.DataFrame(P_to_load_dframe)


fig = px.bar(P_to_load_df, x = V_steps,
             y = P_load_list,
             title = '_________Power Transfered From The Genset To Load.________',
             height = 400,
             width = 1350,
             labels = dict(x = "Time Step (h)", y = "Power(kw)"), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, P_load_list)

$\newline $
### Realised power to charge the battery.

In [ ]:
P_to_bat_dframe = {
             "Power from Genset to battery.": P_to_bat_list,
             'Scale' : P_to_bat_list}

P_to_bat_df = pd.DataFrame(P_to_bat_dframe)

fig = px.bar(P_to_bat_df, x = V_steps,
             y = P_to_bat_list,
             title = '_______Power Transfered From Genset To Battery.________',
             height = 400,
             width = 1350,
             labels = dict(x = "Time Step (h)", y = "Power(kw)", ), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, P_to_bat_list)

$\newline $
### Realised power from the battery to load.

In [ ]:
P_From_bat_dframe = {
             "Power from the battery to load.": P_From_bat_list,
             'Scale' : P_From_bat_list}

P_From_df = pd.DataFrame(P_From_bat_dframe)

fig = px.bar(P_From_df, x = V_steps,
             y = P_From_bat_list,
             title = '_________Power Realised From The Battery To Load._________',
             height = 400,
             width = 1350,
             labels = dict(x = "Time Step (h)", y = "Power(kw)"), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, P_From_bat_list)

### Battery charge balance.

In [ ]:
Q_bat_dframe = {
             'Battery charge balance.': Q_bat_list,
             'Scale' : FC_list}

Q_bat_df = pd.DataFrame(Q_bat_dframe)


fig = px.bar(Q_bat_df, x = V_steps,
             y = Q_bat_list,
             title = '_________Battery Charge Balance._________',
             height = 400,
             width = 1350,
             labels = dict(x = "Time Step (h)", y = "Energy (kwh)"), text_auto = True, color = 'Scale')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, Q_bat_list)

$\newline $
### Genset fuel consumption.

In [ ]:
fc_dframe = {
             'Power generated by the Genset.': P_list,
             'Scale' : FC_list}

fc_df = pd.DataFrame(fc_dframe)
fig = px.bar(fc_df, x = V_steps,
             y = FC_list,
             title = "_________Genset Fuel Comsumption__________",
             height = 400, width = 1350,
             labels = dict(x = "Time Step (h)", y = "FC (g)"), text_auto = True, color = 'Scale',)
#fig.write_image('P_A_list.svg')
Image(fig.to_image(format='png', width=1500, height=350, scale=3))

In [ ]:
print(V_steps, FC_list)

In [ ]:
print(sum(FC_list)*dt)

$\newline $
$\newline $
### Trip power flow bar chart.

In [ ]:
dictframe = {
             'Power generated by the Genset.': P_list,
             "Power from the Genset to load.": P_load_list,
             "Power from the battery to load.": P_From_bat_list,
             "Power load requirements.": d_load_list
               }

df = pd.DataFrame(dictframe)
fig = px.bar(df, x = V_steps,
             y = df.columns,
             barmode = 'group',
             width = 1500,
             text_auto = True,
             title = '________Typical Trip Power Flow._________',
             labels = {'x':'Time step (h)',
                      'value': 'Power (Kw)',
                      'variable': 'Source of power'})
Image(fig.to_image(format='png', width=1500, height=350, scale=3))